In [1]:
import matplotlib
import re
from collections import deque

In [2]:
dimension = 6

identity = {'e': tuple([i for i in range(1, dimension + 1)])}

In [3]:
identity

{'e': (1, 2, 3, 4, 5, 6)}

In [4]:
#permutations = {'r': [(1,2,3), (4,5,6)], 'b': [(1,4),(2,5),(3,6)]} # generates S3 - symmetric
#permutations = {'a': [(1,2,3,4,5,6)]}                              # generates C6 - cyclic
#permutations = {'r': [(1,3,5)], 'b': [(1,2),(3,4)]}                # generates A5 - alternating    https://brauer.maths.qmul.ac.uk/Atlas/alt/A5/
#permutations = {'a': [(1,2,3,4,5,6)], 'b': [(1,2)]}                # generates S6 symmetric
permutations  = {'r': [(1,2,3,4,5,6)], 'f':[(2,6),(3,5)]}           # generates D6

In [5]:
def pretty_print(name):
    pretty_name = ''
    prior, count = '', 1
    for action in name:
        if action != prior:
            if count>1: pretty_name += str(count)
            pretty_name += action           
            prior, count = action, 1
        else:
            count = count + 1
    if count > 1:
        pretty_name +=  str(count)
    
    return pretty_name



In [6]:
# Creates a group based on cyclic permutations
group = dict()
group.update(identity)
new = True
while new:
    new, new_members = False, dict()
    for member_name, member_permutation in group.items(): 
        
        for name, permutes in permutations.items():
            orig_permutation = list(member_permutation)
            work_permutation = list(member_permutation)
            new_name = (name + member_name).replace('e','')
            for permute in permutes:
                sublist = deque(permute)
                sublist.rotate(1)
                for i, element in enumerate(sublist):
                    work_permutation[permute[i] - 1] = orig_permutation[sublist[i] -1]
           
            if (tuple(work_permutation) not in [permutation for permutation in group.values()]) and (tuple(work_permutation) not in [permutation for permutation in new_members.values()]):
                new_members[new_name] = tuple(work_permutation)
        
    if len(new_members) > 0:  
        new = True
        group.update(new_members)     

                   
           



In [7]:
# Takes 2 members of the group, multiplies them together and returns the product

def group_multiply(member1, member2):
    perm1, perm2 = list(group[member1]), list(group[member2])
    result = tuple([perm1[i-1] for i in perm2])
    for key, value in group.items():
        if value == result:
            return key
    
    

In [8]:
# Takes 2 members of a group and generates the coset of the action from the start. If the start is the identity it returns the sub-group

def coset(actions, start='e'):
    coset = set()
    member = start
    
    for action in actions:
        incomplete = True
        while incomplete:
            coset.add(member)
            member = group_multiply(member, action)
            if member == start: incomplete = False
    return frozenset(coset)
    

In [9]:
for row in group.keys():
    for column in group.keys():
        print(pretty_print(group_multiply(row, column)), end='\t')
    print()

e	r	f	r2	fr	rf	r3	fr2	r2f	frf	r4	fr3	
r	r2	fr	r3	fr2	f	r4	fr3	rf	e	frf	r2f	
f	rf	e	r2f	frf	r	fr3	r4	r2	fr	fr2	r3	
r2	r3	fr2	r4	fr3	fr	frf	r2f	f	r	e	rf	
fr	f	r	rf	e	r2	r2f	frf	r3	fr2	fr3	r4	
rf	r2f	frf	fr3	r4	e	fr2	r3	r	f	fr	r2	
r3	r4	fr3	frf	r2f	fr2	e	rf	fr	r2	r	f	
fr2	fr	r2	f	r	r3	rf	e	r4	fr3	r2f	frf	
r2f	fr3	r4	fr2	r3	frf	fr	r2	e	rf	f	r	
frf	e	rf	r	f	r2f	r2	fr	fr3	r4	r3	fr2	
r4	frf	r2f	e	rf	fr3	r	f	fr2	r3	r2	fr	
fr3	fr2	r3	fr	r2	r4	f	r	frf	r2f	rf	e	


In [10]:
len(group)

12

In [11]:
group

{'e': (1, 2, 3, 4, 5, 6),
 'r': (6, 1, 2, 3, 4, 5),
 'f': (1, 6, 5, 4, 3, 2),
 'rr': (5, 6, 1, 2, 3, 4),
 'fr': (6, 5, 4, 3, 2, 1),
 'rf': (2, 1, 6, 5, 4, 3),
 'rrr': (4, 5, 6, 1, 2, 3),
 'frr': (5, 4, 3, 2, 1, 6),
 'rrf': (3, 2, 1, 6, 5, 4),
 'frf': (2, 3, 4, 5, 6, 1),
 'rrrr': (3, 4, 5, 6, 1, 2),
 'frrr': (4, 3, 2, 1, 6, 5)}

In [12]:
for member in group.keys():
    name = pretty_print(member)
    print(name, member, len(name))


e e 1
r r 1
f f 1
r2 rr 2
fr fr 2
rf rf 2
r3 rrr 2
fr2 frr 3
r2f rrf 3
frf frf 3
r4 rrrr 2
fr3 frrr 3


In [13]:
cosets = set()
for member1 in group:
    for member2 in group:
        subgroup = coset([member1, member2])   
        if subgroup not in cosets:
            cosets.add(subgroup)
            pretty_subgroup = [pretty_print(member) for member in subgroup]
            print(pretty_print(member), pretty_print(member2), pretty_subgroup)
   
    

fr3 e ['e']
fr3 r ['r4', 'frf', 'r', 'r2', 'r3', 'e']
fr3 f ['f', 'e']
fr3 r2 ['r4', 'e', 'r2']
fr3 fr ['fr', 'e']
fr3 rf ['rf', 'e']
fr3 r3 ['r3', 'e']
fr3 fr2 ['fr2', 'e']
fr3 r2f ['r2f', 'e']
fr3 fr3 ['e', 'fr3']
fr3 f ['f', 'r4', 'frf', 'r', 'r2', 'r3', 'e']
fr3 fr ['r4', 'frf', 'r', 'r2', 'r3', 'fr', 'e']
fr3 rf ['rf', 'r4', 'frf', 'r', 'r2', 'r3', 'e']
fr3 fr2 ['r4', 'frf', 'r', 'r2', 'fr2', 'r3', 'e']
fr3 r2f ['r4', 'frf', 'r', 'r2', 'r3', 'r2f', 'e']
fr3 fr3 ['r4', 'frf', 'r', 'fr3', 'r2', 'r3', 'e']
fr3 r2 ['r4', 'f', 'e', 'r2']
fr3 fr ['f', 'fr', 'e']
fr3 rf ['rf', 'f', 'e']
fr3 r3 ['f', 'e', 'r3']
fr3 fr2 ['fr2', 'f', 'e']
fr3 r2f ['f', 'r2f', 'e']
fr3 fr3 ['f', 'e', 'fr3']
fr3 fr ['r4', 'fr', 'e', 'r2']
fr3 rf ['rf', 'r4', 'e', 'r2']
fr3 r3 ['r4', 'r3', 'e', 'r2']
fr3 fr2 ['fr2', 'r4', 'e', 'r2']
fr3 r2f ['r4', 'r2f', 'e', 'r2']
fr3 fr3 ['r4', 'e', 'fr3', 'r2']
fr3 rf ['rf', 'fr', 'e']
fr3 r3 ['r3', 'fr', 'e']
fr3 fr2 ['fr2', 'fr', 'e']
fr3 r2f ['r2f', 'fr', 'e']
fr3 fr3 ['

In [14]:
len(cosets)

44